In [ ]:
#Las librerías utilizadas en este documento son:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn import metrics
from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
sns.set()

In [ ]:
#leemos el df creado para el barrio de caballito
df = pd.read_csv('./data/data_limpio_gdf_caballito.csv')

## Regresión lineal múltiple

Agregamos a la correlacion las variables latitud y longitud

In [ ]:
# Asignamos las variables predictoras

X = df[['lat', 'lon', 'sup_m2_total']]

y = df['precio_usd']

# Normalizamos los datos

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# Dividimos en train y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instanciamos el modelo y lo entrenamos

lr= linear_model.LinearRegression()
lr.fit(X_train, y_train)

# Vemos los coeficientes

print('Coeficientes: ', lr.coef_)
print('Intercepto: ', lr.intercept_)
print('R2_train: ', r2_score(y_train, lr.predict(X_train)))
print('R2_test: ', r2_score(y_test, lr.predict(X_test)))
print ('MSE:', metrics.mean_squared_error(y, lr.predict(X)))
rmse_multi = np.sqrt(metrics.mean_squared_error(y, lr.predict(X)))
print ('rMSE:', rmse_multi)


In [ ]:
rmse_simple = 113040.74611014882

print('Diferencia porcentual entre el rmse de la regresión simple y la regresión múltiple: ', (rmse_multi - rmse_simple)/rmse_simple*100)
print('Diferencia absoluta entre el rmse de la regresión simple y la regresión múltiple: ', rmse_multi - rmse_simple)

Recordemos que para la regresión lineal simple, la raíz del error cuadrático medio (RMSE) fue de `r_EMC: 113040.74611014882`, muy similar al RMSE de la regresión lineal múltiple, que es de `r_EMC: 124196.721423661`, una diferencia de `r_EMC: 11155.97531351218` que es del 10% del RMSE de la regresión lineal simple.

Esto nos indica que la regresión lineal múltiple no mejora el modelo, por lo que no es necesario agregar más variables.

Podemos también confirmar que la regresión lineal múltiple no mejora el modelo viendo la no mejora del score de validación cruzada y las correlaciones de las variables.

In [ ]:
# Cross validation
scores = cross_val_score(lr, X, y, cv=5)

print('Cross-validated scores:', scores)
print('Cross-validated scores mean:', scores.mean())


In [ ]:
# Observamos las correlaciones entre las variables

corr = df[['lat', 'lon', 'sup_m2_total', 'precio_usd']].corr()

# Graficamos la matriz de correlaciones

plt.figure(figsize=(5,5))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.2)
plt.title('Matriz de correlaciones', fontsize=20)
plt.show()

In [ ]:
# Modelamos con statsmodels

X_train_sm = sm.add_constant(X_train)

model = sm.OLS(y_train, X_train_sm).fit()

print(model.summary())

Comparando la regresión multiple sencilla con la regresión OLS podemos observar como OLS nos permite validar la significancia de los datos obtenidos. Vemos con los p values para las variables latitud y longitud son mayores al nivel de significancia por lo cual estas variables no están explicando o no tiene valor de predicción sobre el valor de nuestra variable objetivo precio en dólares.

Al incluir latitud y longitud disminuye el performance del R2 a 0,38 en la regresión multiple.